# Generator of C lang samples (all of them I guess)

In [ ]:
import numpy as np
import random
import math

## version 1.0

There is always type, an ID and openning curly brace at the beginning that simulates a function definition. Function can have parameters.


While generating I assume that if some ID is defined than it is defined "in order", it means that there should not be a situation when ID2 is defined before ID1 has been defined


# List of things that can be potentially included (TODO list)

* usage of pointers
* getting rid of STRINGs
* add floats
* add parenthesis in expressions
* choice frequency:
  * more definitions at the begginning

In [ ]:
beg_call = "("
end_call = ")"
beg_block = "{"
end_block = "}"

built_in_types = ["int", "char"]
types_to_return_from_function = built_in_types + ["void"]

math_ops = ["-", "+", "/", "*"]
EMPTY = "EMPTY"
static_values_tokens = ["STRING", "NUM"]

vocabulary = [
  "void", "int", "char", "STRING", "NUM", "return", "printf", "scanf", 
  "-", "+", "/", "*", "&", "=", "(", ")", ";", "{", "}", "EMPTY", ","
]

IDs = []
funs = []
callable_funs = ["printf", "scanf"] + funs

for index in range(16):
  current_id = "ID" + str(index)
  vocabulary.append(current_id)
  IDs.append(current_id)

# will add functions in some further generation
# for index in range(8):
#   current_fun = "FUN" + str(index)
#   vocabulary.append(current_fun)
#   funs.append(current_fun)

print(len(vocabulary))
print(vocabulary)
print(IDs)
print(funs)

cv1Dict = {el:idx for idx,el in enumerate(vocabulary)}

In [ ]:
#
# defined_ids is a tuple -> (type, ID, definition_place)
# definition_place function-:
#   0: -body
#   1: -argument
#   2: -name

def val_token_dep_on_type(type):
  if type == "char":
    return "STRING"
  if type == "int":
    return "NUM"
  raise Exception(f"No known type '{type}'")

def get_first_free_ID(defined_ids):
  if len(defined_ids) >= len(IDs):
    raise Exception(f"hueheu, cannot give you another ID cause you already have {len(defined_ids)}")
  return IDs[len(defined_ids)]

def arg_pair(defined_ids):
  type = random.choice(built_in_types)
  return type, (type, get_first_free_ID(defined_ids), 1)

def gen_args(defined_ids, no_of_args):
  args = []
  ids = []
  for _ in range(no_of_args):
    type, id = arg_pair(defined_ids + ids)
    ids.append(id)
    args += [type, id[1]] + [","]

  # removing trailing comma (',')
  if len(args):
    del args[-1]

  return args, ids

def function_start():
  function_type = random.choice(types_to_return_from_function)
  to_return = [function_type, IDs[0], "("]
  defined_ids = [(function_type, IDs[0], 2)]
  no_of_args = random.randint(0, len(IDs) // (len(IDs) ** 0.5))
  args, ids = gen_args(defined_ids, no_of_args)
  
  defined_ids += ids
  to_return += args + [")", "{"]
  return to_return, defined_ids

# will add functions in some further generation
# returns list of ID that are passed as arguments
def call(defined_ids):
  to_return = []
  calls_choices = ["printf", "scanf"] # for now and this whole function will change when I add function calls (or just expression function to use here)
  chosen_call = random.choice(calls_choices)
  chosen_value = random.choice(static_values_tokens + [id[1] for id in defined_ids if id[2] != 2])
  to_return += [chosen_call, "(", "STRING", ",", chosen_value, ")", ";"]
  return to_return, []

# can be called only if there are any IDs defined already (not as arguments)
# this function is probably going to be the most frequently changed part of this generation
def assignment(defined_ids):
  chosen_type, chosen_id, definition_place = random.choice(defined_ids)
  if definition_place == 0:
    return [], [] # cycling
  while(chosen_type == "void"):
    if len(defined_ids) == 1:
      return definition(defined_ids)
    chosen_type, chosen_id, definition_place = random.choice(defined_ids)
  return [chosen_id, "=", *(expression(defined_ids, chosen_type, [chosen_id])[0]), ";"], []

# returns ID that has been defined
def definition(defined_ids):
  if len(defined_ids) >= len(IDs):
    return [], [] # cycling
  id = get_first_free_ID(defined_ids)
  type = random.choice(built_in_types)
  if random.random() > 0.5: # no assignment
    tokens = [type, id, ";"]
  else:
    tokens = [type, id, "=", *(expression(defined_ids, type, [id])[0]), ";"]
  return tokens, [(type, id, 1)]


# the idea of this function is that it generates a sequence of tokens that will be used in other actions
def expression(defined_ids, of_type, excluded_ids):
  length = random.randint(1, 6)
  # if I would want to add recursion than should remove condition of excluding main function
  possible_vals = [id[1] for id in defined_ids if (id != "ID0" and id[0] == of_type and id[1] not in excluded_ids)] # ids
  if of_type == "int":
    val = "NUM"
  else:
    val = "STRING"
  possible_vals += [val]
  operations = [random.choice(math_ops) for _ in range(length - 1)]

  # merging 
  to_return = []
  for op in operations:
    to_return += [random.choice(possible_vals), op]
  to_return.append(possible_vals[-1])
  return to_return, []

def calculate_choice_weights(defined_ids, counts):
  definition_w = (len(IDs) - len(defined_ids)) ** 1.1 / counts[1]
  call_w = len(callable_funs)
  # have fun with this static value
  # static * (all actions count - count of assignments) / num of counts
  assignment_w = float(3 * (sum(counts) - counts[2])) / len(counts)
  return [call_w, definition_w, assignment_w]


def gen_actions(defined_ids, no_of_actions):
  to_return = []

  counts = [1,1,1]
  actions = [call, definition, assignment]
  for _ in range(no_of_actions):
    weights = calculate_choice_weights(defined_ids, counts)
    # print(weights)
    action = random.choices(actions, weights)[0]
    # keep track of how many of them there is
    if action is call:
      counts[0] += 1
    elif action is definition:
      counts[1] += 1
    elif action is assignment:
      counts[2] += 1
    tokens, new_ids = action(defined_ids)
    to_return += tokens
    defined_ids += new_ids
    
  return to_return, defined_ids

def single_sample():
  to_return = []
  no_of_actions = random.randint(8, 27)
  tokens, defined_ids = function_start()
  to_return += tokens
  to_return += gen_actions(defined_ids, no_of_actions)[0]
  to_return += ["return", "NUM", ";", "}"] # end of function
  return to_return

def c_v1_simple_generate_samples(no_of_samples):
  to_return = []
  for _ in range(no_of_samples):
    to_return.append(single_sample())
  return to_return


def fill_vector_with_empty_tokens(sample, length):
  if len(sample) > length:
    print("Sample is too long already. Yeeting it out of existence")
    return []
  for _ in range(length - len(sample)):
    sample.append("EMPTY")
  return sample

def tokens_to_vals(sample):
  return [cv1Dict[token] for token in sample]

In [ ]:
generated = c_v1_simple_generate_samples(1)[0]
print(len(generated))
print(generated)
print(" ".join(generated).replace(";", ";\n").replace("{", "{\n"))
print(tokens_to_vals(generated))

In [ ]:
num_of_samples_to_generate = 10_000
dataset = c_v1_simple_generate_samples(num_of_samples_to_generate)
lens = [len(sample) for sample in dataset]
max_len, min_len, sum_len = max(lens), min(lens), sum(lens)
avg_lev = sum_len / len(dataset)

print(max_len)
print(min_len)
print(avg_lev)
target_length = 256
below_target_length = [l for l in lens if l < target_length]
print(len(below_target_length))
print(num_of_samples_to_generate - len(below_target_length))

In [ ]:
converted_dataset_within_target_length = [tokens_to_vals(fill_vector_with_empty_tokens(sample, target_length)) for sample in dataset if len(sample) < target_length]
print(len(converted_dataset_within_target_length))
# fill samples to full length
final_dataset = np.asarray(converted_dataset_within_target_length)
print(final_dataset.shape)

## save dataset

In [ ]:
data_dir_path = "..\\data\\simple_c_v1"
for idx, sample in enumerate(final_dataset):
  np.savetxt(f"{data_dir_path}\\sample_no{idx}.txt", sample, newline=" ", fmt="%u")